In [1]:
# 1. 필요한 라이브러리 임포트
import time
from datetime import datetime
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import requests
import json
import os
from datetime import datetime
import mysql.connector
from mysql.connector import Error

In [2]:
# # 2. 웹드라이버 설정
# def setup_driver():
#     chrome_options = Options()
#     chrome_options.add_argument('--disable-blink-features=AutomationControlled')
#     chrome_options.add_argument('--disable-gpu')
#     chrome_options.add_argument('--no-sandbox')
#     chrome_options.add_argument('--disable-dev-shm-usage')
#     chrome_options.add_argument('--window-size=1920,1080')
#     chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    
#     driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), 
#                             options=chrome_options)
#     return driver


In [3]:
# 2. 웹드라이버 설정
def setup_driver():
    chrome_options = Options()
    
    # 리눅스 환경에서의 추가 설정
    chrome_options.add_argument('--headless')  # 헤드리스 모드 실행
    chrome_options.add_argument('--no-sandbox')  # 샌드박스 비활성화
    chrome_options.add_argument('--disable-dev-shm-usage')  # 공유 메모리 사용 비활성화
    chrome_options.add_argument('--disable-gpu')  # GPU 하드웨어 가속 비활성화
    
    # 기존 설정들
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

    # 크롬 드라이버 생성 시 에러 처리
    try:
        driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=chrome_options
        )
        return driver
    except Exception as e:
        print(f"드라이버 설정 중 오류 발생: {e}")
        
        # 대체 방법 시도
        try:
            print("대체 방법으로 드라이버 설정 시도...")
            chrome_options.add_argument('--remote-debugging-port=9222')
            driver = webdriver.Chrome(
                options=chrome_options
            )
            return driver
        except Exception as sub_e:
            print(f"대체 방법도 실패: {sub_e}")
            raise

In [4]:
# 3. 공고 상세 페이지 크롤링 함수
def get_announcement_detail(url, headers):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        board_view = soup.find('div', class_='board_view')
        
        if not board_view:
            return None
            
        data = {
            'POSTDATE': None,
            'ANNOUNCEMENT_NUMBER': None,
            'TITLE': None,
            'CATEGORY': None,
            'LOCATION': "전국",
            'CONTENT': None,
            'START': None,
            'END': None,
            'AGENCY': None,
            'LINK': url,
            'FILE': None,
            'KEYWORD': None
        }
        
        # 제목
        title_tag = board_view.find('h4')
        if title_tag:
            data['TITLE'] = title_tag.text.strip()
        
        # 테이블 데이터 추출
        table = board_view.find('table')
        if table:
            rows = table.find_all('tr', class_='web')
            for row in rows:
                ths = row.find_all('th')
                tds = row.find_all('td')
                for th, td in zip(ths, tds):
                    key = th.text.strip()
                    value = td.text.strip()
                    
                    if key == '공고번호':
                        data['ANNOUNCEMENT_NUMBER'] = value
                    elif key == '신청기간':
                        periods = value.split('~')
                        if len(periods) >= 1:
                            start_date = extract_date(periods[0])
                            if start_date:
                                data['START'] = start_date
                        
                        if len(periods) >= 2:
                            end_date = extract_date(periods[1])
                            if end_date:
                                data['END'] = periods[1].strip()
                            else:
                                data['END'] = periods[1].strip()
                    elif key == '담당부서':
                        data['AGENCY'] = value
                    elif key == '등록일':
                        data['POSTDATE'] = extract_date(value)
        
        # 내용
        content_div = board_view.find('div', class_='txt-area')
        if content_div:
            content_text = content_div.get_text(separator=' ')  # 태그 사이에 공백 추가
            data['CONTENT'] = clean_content(content_text)

        # 첨부파일
        files = []
        file_list = board_view.find('td', class_='file_list')
        if file_list:
            for file_item in file_list.find_all('li'):
                file_info = {}
                name_tag = file_item.find('span', class_='name')
                if name_tag:
                    file_info = {
                        'name': name_tag.text.strip()
                    }
                    files.append(file_info)

        data['FILE'] = clean_file_info(files)
        
        # 키워드 추출
        #keyword_row = board_view.find('th', text='키워드')
        keyword_row = board_view.find('th', string='키워드')  # text 대신 string 사용
        if keyword_row:
            keyword_cell = keyword_row.find_next('td')
            if keyword_cell:
                data['KEYWORD'] = keyword_cell.text.strip()
        
        # 지역 정보 추출
        location_text = f"{data['TITLE']} {data['CONTENT']} {data['AGENCY']}"
        data['LOCATION'] = extract_location(location_text)
        
        return data
        
    except Exception as e:
        print(f"상세 페이지 크롤링 중 오류 발생: {e}")
        return None

In [5]:
def get_announcement_list(base_url, start_page=1, end_page=1):
    # 체크포인트 초기화
    checkpoint = CrawlerCheckpoint('mss')
    announcements = []
    stop_crawling = False
    
    try:
        page = start_page
        while end_page is None or page <= end_page: # end_page가 None이면 계속 진행
            if stop_crawling:
                break
                
            print(f"\n{'='*50}")
            print(f"현재 {page}페이지 크롤링 시작")
            
            url = f"{base_url}&pageIndex={page}"
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            table = soup.find('table')
            if table and table.find('tbody'):
                rows = table.find('tbody').find_all('tr')
                
                for idx, row in enumerate(rows, 1):
                    try:
                        reg_date = row.find_all('td')[6].text.strip()  # 등록일
                        title = row.find('td', class_='subject').find('a').text.strip()
                        
                        # checkpoint 파일이 있는 경우 비교
                        if checkpoint.last_crawled:
                            last_date = checkpoint.last_crawled['last_post_date']
                            last_title = checkpoint.last_crawled['last_title']
                            
                            print(f"현재 게시물: {reg_date}, {title}")
                            print(f"저장된 게시물: {last_date}, {last_title}")
                            
                            if reg_date == last_date and title == last_title:
                                print(f"\n이전 수집 지점 도달. 크롤링 중단")
                                stop_crawling = True
                                break
                        
                        # 첫 번째 게시물이면서 첫 페이지인 경우 체크포인트 업데이트
                        if page == 1 and idx == 1:
                            print(f"체크포인트 업데이트: {reg_date}, {title}")
                            checkpoint.save_checkpoint(reg_date, title)
                        
                        # 게시물 상세 정보 수집
                        link_tag = row.find('td', class_='subject').find('a')
                        if link_tag:
                            onclick = link_tag.get('onclick', '')
                            match = re.search(r"doBbsFView\('(\d+)',\s*'(\d+)'", onclick)
                            if match:
                                cbIdx, bcIdx = match.groups()
                                detail_url = f"https://www.mss.go.kr/site/smba/ex/bbs/View.do?cbIdx={cbIdx}&bcIdx={bcIdx}&parentSeq={bcIdx}"
                                
                                announcement_data = get_announcement_detail(detail_url, headers)
                                if announcement_data:
                                    announcements.append(announcement_data)
                                    
                    except Exception as e:
                        print(f"게시물 처리 중 오류: {e}")
                        continue
                        
                    time.sleep(1)
            
            print(f"{page}페이지 완료 - {len(announcements)}건 수집")
            time.sleep(2)

            # 더 이상 데이터가 없으면 종료
            if not rows:  # 페이지에 데이터가 없으면
                break
                
            page += 1
            
    except Exception as e:
        print(f"크롤링 중 오류 발생: {e}")
    
    return announcements

In [6]:
def extract_date(date_str):
    """날짜 문자열을 YYYY-MM-DD 형식으로 변환"""
    if not date_str:
        return None
    try:
        # 다양한 날짜 형식 처리
        date_str = date_str.replace('.', '-').strip()
        if len(date_str) == 10:  # YYYY-MM-DD
            return date_str
        return None
    except:
        return None

In [7]:
def clean_content(text):
   """HTML 태그 및 특수문자 제거"""
   if not text:
       return text
       
   # HTML 태그 제거
   text = re.sub(r'<[^>]+>', '', text)
   
   # HTML 특수문자 변환 및 제거
   html_chars = {
       '&nbsp;': ' ',
       '&#39;': "'",
       '&quot;': '"', 
       '&lt;': '<',
       '&gt;': '>',
       '&amp;': '&',
       '&middot;': '',  # middot 제거
       '&bull;': '',    # 글머리 기호 제거
       '&rarr;': '',    # 화살표 제거
       '&raquo;': '',   # 이중 화살표 제거
       '&laquo;': '',   # 이중 화살표 제거
       '&ndash;': '-',  # 대시
       '&mdash;': '-',  # 대시
   }
   
   for char, replace in html_chars.items():
       text = text.replace(char, replace)
   
   # 나머지 HTML 엔티티 제거 (&로 시작하는 모든 특수문자)
   text = re.sub(r'&[a-zA-Z0-9#]+;', '', text)
   
   # 공고 번호 형식의 텍스트 제거
   text = re.sub(r'중소벤처기업부 공고 제\d{4}-\d+호', '', text)
   
   # 연속된 공백 제거
   text = re.sub(r'\s+', ' ', text)
   
   # 앞뒤 공백 제거
   text = text.strip()
   
   return text

In [8]:
def clean_file_info(files):
   """파일 정보에서 파일명만 추출하고 문자열로 변환"""
   if not files:
       return None
       
   file_names = []
   for file in files:
       if 'name' in file:
           # 파일명과 크기 분리
           name = file['name'].split('  ')[0]  # 파일 크기 정보 제거
           file_names.append(name)
   
   # 리스트를 문자열로 변환 (쉼표로 구분)
   return ', '.join(file_names) if file_names else None

In [9]:
def extract_location(text):
    """텍스트에서 지역 정보 추출"""
    locations = ['서울', '경기', '인천', '강원', '충북', '충남', '대전', '세종', 
                '전북', '전남', '광주', '경북', '경남', '대구', '울산', '부산', '제주']
    
    location_count = {}
    for loc in locations:
        count = text.count(loc)
        if count > 0:
            location_count[loc] = count
    
    if location_count:
        # 가장 많이 언급된 지역 반환
        return max(location_count.items(), key=lambda x: x[1])[0]
    return "전국"  # 기본값


In [10]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

class CrawlerCheckpoint:
    def __init__(self, site_name):
        self.site_name = site_name
        self.checkpoint_file = f'checkpoints/{site_name}_last_crawled.json'
        self.last_crawled = self.load_checkpoint()
        
    def load_checkpoint(self):
        """체크포인트 파일 로드"""
        if not os.path.exists('checkpoints'):
            os.makedirs('checkpoints')
            
        if os.path.exists(self.checkpoint_file):
            with open(self.checkpoint_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return None
        
    def save_checkpoint(self, post_date, title):
        """최신 크롤링 정보 저장"""
        checkpoint_data = {
            'last_post_date': post_date,
            'last_title': title,
            'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        with open(self.checkpoint_file, 'w', encoding='utf-8') as f:
            json.dump(checkpoint_data, f, ensure_ascii=False, indent=2)
        
        self.last_crawled = checkpoint_data

In [11]:
# 체크포인트 확인을 위한 함수 추가
def print_checkpoint(site_name='mss'):
    checkpoint = CrawlerCheckpoint(site_name)
    if checkpoint.last_crawled:
        print("\n현재 저장된 체크포인트:")
        print(f"마지막 수집 날짜: {checkpoint.last_crawled['last_post_date']}")
        print(f"마지막 수집 제목: {checkpoint.last_crawled['last_title']}")
        print(f"업데이트 시간: {checkpoint.last_crawled['updated_at']}")
    else:
        print("\n저장된 체크포인트가 없습니다.")


In [12]:
def connect_to_database():
    try:
        connection = mysql.connector.connect(
            host='10.100.54.176',          # DB 호스트
            database='ALRIMI',         # DB 이름
            user='root',      # DB 사용자명
            password='ibdp',   # DB 비밀번호
            charset='utf8mb4',
            collation='utf8mb4_general_ci'
        )
        return connection
    except Error as e:
        print(f"DB 연결 오류: {e}")
        return None

In [25]:
def insert_into_db(connection, announcements):
    try:
        cursor = connection.cursor()
        
        insert_query = """
            INSERT INTO Crawler (
                POSTDATE, ANNOUNCEMENT_NUMBER, TITLE, 
                CATEGORY, LOCATION, CONTENT, 
                START, END, AGENCY, 
                LINK, FILE, KEYWORD
            ) VALUES (
                %s, %s, %s, %s,
                %s, %s, %s, %s, 
                %s, %s, %s, %s
            )
        """
        
        for announcement in announcements:
            values = (
                announcement.get('POSTDATE'),
                announcement.get('ANNOUNCEMENT_NUMBER'),
                announcement.get('TITLE'),
                announcement.get('CATEGORY'),
                announcement.get('LOCATION'),
                announcement.get('CONTENT'),
                announcement.get('START'),
                announcement.get('END'),
                announcement.get('AGENCY'),
                announcement.get('LINK'),
                announcement.get('FILE'),
                announcement.get('KEYWORD')
            )
            
            cursor.execute(insert_query, values)
        
        connection.commit()
        print(f"{len(announcements)}개의 공고가 DB에 저장되었습니다.")
        
    except Error as e:
        print(f"데이터 저장 중 오류 발생: {e}")
        connection.rollback()
    
    finally:
        if connection.is_connected():
            cursor.close()


In [31]:
# 메인 실행 코드 수정
def main():
    # 실행 전 체크포인트 확인
    print_checkpoint()
    
    print("크롤링 시작...")
    base_url = "https://www.mss.go.kr/site/smba/ex/bbs/List.do?cbIdx=310&searchRltnYn=A"
    
    # 데이터 수집
    announcements = get_announcement_list(base_url, start_page=1, end_page=1)
    
    print("\n최종 결과:")
    print(f"총 {len(announcements)}개의 공고 수집 완료")
    
    # DB 연결 및 데이터 저장
    connection = connect_to_database()
    if connection:
        insert_into_db(connection, announcements)
        connection.close()
    
    # DataFrame으로 변환 (확인용)
    df = pd.DataFrame(announcements)
    display(df)

    
    # 크롤링 후 체크포인트 확인
    print_checkpoint()

if __name__ == "__main__":
    main()


저장된 체크포인트가 없습니다.
크롤링 시작...

현재 1페이지 크롤링 시작
체크포인트 업데이트: 2024.11.08, 2025년 중소기업 혁신바우처 사업 지원계획 공고문
현재 게시물: 2024.11.01, 소상공인 전기요금 특별지원사업 시행 수정공고
저장된 게시물: 2024.11.08, 2025년 중소기업 혁신바우처 사업 지원계획 공고문
현재 게시물: 2024.10.29, 벤처투자회사 등록말소
저장된 게시물: 2024.11.08, 2025년 중소기업 혁신바우처 사업 지원계획 공고문
현재 게시물: 2024.10.25, 2024년 소상공인 정책자금 융자계획 변경공고
저장된 게시물: 2024.11.08, 2025년 중소기업 혁신바우처 사업 지원계획 공고문
현재 게시물: 2024.10.25, 2024년 중소기업 정책자금 융자계획 변경공고
저장된 게시물: 2024.11.08, 2025년 중소기업 혁신바우처 사업 지원계획 공고문
현재 게시물: 2024.10.24, (통합공고) 위메프·티몬·인터파크·AK몰·알렛츠 정산지연 피해 소상공인·중소기업에 대한 이커머스 플랫폼 입점지원
저장된 게시물: 2024.11.08, 2025년 중소기업 혁신바우처 사업 지원계획 공고문
현재 게시물: 2024.10.24, 스케일업 팁스 운영사 모집 공고(연장)
저장된 게시물: 2024.11.08, 2025년 중소기업 혁신바우처 사업 지원계획 공고문
현재 게시물: 2024.10.21, 2024년 소상공인 정책자금 융자계획 변경공고
저장된 게시물: 2024.11.08, 2025년 중소기업 혁신바우처 사업 지원계획 공고문
현재 게시물: 2024.10.17, (수정 공고) 위메프·티몬·인터파크·AK몰·알렛츠 정산지연 피해 소상공인에 대한 이커머스 플랫폼 입점지원
저장된 게시물: 2024.11.08, 2025년 중소기업 혁신바우처 사업 지원계획 공고문
현재 게시물: 2024.10.16, 2024년 기술개발제품 시범구매 지원계획 공고(4차)
저장된 게시물: 2024.11.08, 2025년 중소기업 혁신

,POSTDATE,ANNOUNCEMENT_NUMBER,TITLE,CATEGORY,LOCATION,CONTENT,START,END,AGENCY,LINK,FILE,KEYWORD
0,2024-11-08,제2024-567호,2025년 중소기업 혁신바우처 사업 지원계획 공고문,None,전국,2025년 중소기업 혁신바우처 사업 지원계획을 붙임과 같이 공고합니다.,None,,지역혁신정책과,https://www.mss.go.kr/site/smba/ex/bbs/View.do...,"2025년_중소기업_혁신바우처_사업_1차_지원계획_공고.hwpx, 2025년_중소기...",
1,2024-11-01,제2024-557호,소상공인 전기요금 특별지원사업 시행 수정공고,None,전국,2024년 소상공인 전기요금 특별지원 사업을 붙임과 같이 수정 공고합니다.,2024-11-01,,소상공인손실보상과,https://www.mss.go.kr/site/smba/ex/bbs/View.do...,소상공인_전기요금_특별지원_사업_시행_수정공고(제2024-557호).pdf,
2,2024-10-29,중소벤처기업부 공고 제2024-554호,벤처투자회사 등록말소,None,전국,벤처투자회사 등록말소 '벤처투자 촉진에 관한 법률' 제48조에 따라 아래와 같이 벤...,None,,투자관리감독과,https://www.mss.go.kr/site/smba/ex/bbs/View.do...,중소벤처기업부_공고_제2024-554호(벤처투자회사_등록말소).pdf,
3,2024-10-25,제2024-551호,2024년 소상공인 정책자금 융자계획 변경공고,None,전국,붙임과 같이 2024년 소상공인 정책자금 융자계획을 변경하여 공고합니다.,2024-10-28,,기업금융과,https://www.mss.go.kr/site/smba/ex/bbs/View.do...,"2024년_소상공인_정책자금_운용계획(제2024-551호).hwpx, 2024년_소...",
4,2024-10-25,제2024-552호,2024년 중소기업 정책자금 융자계획 변경공고,None,전국,2024년 중소기업 정책자금 융자계획을 붙임과 같이 변경 공고합니다.,None,,기업금융과,https://www.mss.go.kr/site/smba/ex/bbs/View.do...,"2024년_중소기업_정책자금_융자계획_공고(제2024-552호).pdf, 2024년...",
5,2024-10-24,중소벤처기업부 공고 제2024-546호,(통합공고) 위메프·티몬·인터파크·AK몰·알렛츠 정산지연 피해 소상공인·중소기업에 ...,None,전국,ㅇ위메프티몬인터파크쇼핑AK몰알렛츠의 판매대금 정산지연으로 피해를 입은 소상공인중소기...,2024-08-28,,디지털소상공인과,https://www.mss.go.kr/site/smba/ex/bbs/View.do...,(통합공고)_위메프·티몬·인터파크·AK몰·알렛츠_정산지연_피해_소상공인·중소기업에_...,
6,2024-10-24,제2024-544호,스케일업 팁스 운영사 모집 공고(연장),None,전국,"투자시장과 연구개발 분야의 민간 전문역량을 활용하여, 국가전략기술 등 주요 기술분야...",2024-10-14,2024-11-01,기술혁신정책과,https://www.mss.go.kr/site/smba/ex/bbs/View.do...,"스케일업_팁스_운영사_모집_공고(연장).hwpx, [첨부1]_2024년_스케일업_팁...",
7,2024-10-21,제2024-541호,2024년 소상공인 정책자금 융자계획 변경공고,None,전국,2024년 소상공인 정책자금 융자계획을 붙임과 같이 변경하여 공고합니다.,2024-10-21,,기업금융과,https://www.mss.go.kr/site/smba/ex/bbs/View.do...,"2024년_소상공인_정책자금_운용계획_변경공고(제2024-541호).hwpx, 20...",
8,2024-10-17,제2024 - 538호,(수정 공고) 위메프·티몬·인터파크·AK몰·알렛츠 정산지연 피해 소상공인에 대한 이...,None,전국,"위메프, 티몬, 인터파크쇼핑, AK몰, 알렛츠의 판매대금 정산 지연으로 피해를 입은...",2024-08-28,,디지털소상공인과,https://www.mss.go.kr/site/smba/ex/bbs/View.do...,(수정_공고)_위메프·티몬·인터파크·AK몰·알렛츠_정산지연_피해_소상공인에_대한_이...,
9,2024-10-16,제2024-536호,2024년 기술개발제품 시범구매 지원계획 공고(4차),None,전국,2024년 기술개발제품 시범구매 지원계획 공고(4차),None,,판로정책과,https://www.mss.go.kr/site/smba/ex/bbs/View.do...,"(공고)_2024년_기술개발제품_시범구매_지원계획_공고(안)(4차).hwpx, (별...",



현재 저장된 체크포인트:
마지막 수집 날짜: 2024.11.08
마지막 수집 제목: 2025년 중소기업 혁신바우처 사업 지원계획 공고문
업데이트 시간: 2024-11-10 13:40:46


In [15]:
# CSV 파일로 저장
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = f"announcements_{current_time}.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')  # utf-8-sig는 한글 깨짐 방지
print(f"\nCSV 파일이 생성되었습니다: {csv_filename}")


NameError: name 'df' is not defined